# Google Colab

You can use the button below to open this notebook in Google Colab. Note that changes made to the notebook in Colab will not be reflected in Github, nor can the notebook be saved on Colab without first making a copy. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nikitalokhmachev-ai/radio-map-estimation-public/blob/main/notebooks/Visualize_Results.ipynb)

If opened in Colab, set `using_colab` to `True` in the code block below, then run the second and (optionally) third blocks. The second block will install kaleido to visualize some of the results, then clone the github repository into Colab's local storage in order to load the models and other functions. The third block will connect to Google Drive (user login required), which allows the Colab notebook to read and write data to the drive (e.g. training data or evaluation results).

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    %cd /content/
    !rm -rf /content/radio-map-estimation-public
    !git clone https://github.com/nikitalokhmachev-ai/radio-map-estimation-public.git
    !pip install -q -r /content/radio-map-estimation-public/requirements.txt

In [ ]:
if using_colab:
    from google.colab import drive
    drive.mount('/content/drive')

# Import Packages

In [ ]:
import torch
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import os
import glob
import pickle

In [ ]:
os.chdir('path/to/repository')
from test_utils import get_sample_error, visualize_sample_error, get_average_error, visualize_average_error, visualize_hist

# Model and Result Paths

Specify paths to the folders where the trained models are saved, where the results are saved, and where the visualizations (graphs) should be saved. For the PIMRC paper, we included some graphs showing all models' performance, and some graphs showing just the performance of Dual Path or UNet models. Below, we specify a single folder for `all_results`, and then two folders for `dual_results` and `unet_results`. One way to accomplish this is to have the Dual Path and UNet result folders saved under an overarching folder for All results, then conclude the path to the All results folder with `\**`, which indicates a recursive search within that folder when using the `glob` library.

In [ ]:
# Specify folder containing trained models
model_folder = '/Path/to/saved/models'

# Specify folder containing all saved results
all_results = '/Path/to/saved/results'

# Specify folder containing Dual Path saved results
dual_results = '/Path/to/dual_path/results'

# Specify folder containing Skip Connection saved results
unet_results = '/Path/to/UNet/results'

# Set folder to save visualizations
viz_folder = '/Path/to/save/visualizations'
if not os.path.exists(viz_folder):
    os.makedirs(viz_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Display Names

This is optional code to specify how model names will appear in visualizations. 

`display_names` is a dictionary, the keys of which are the names that the individual model results are saved under (minus the ".pickle" ending), and the values of which are how they will appear in visualizations. Below are the names of the models as they were saved and appear in the PIMRC paper. `display_names` is provided as an optional parameter to the `visualize_sample_error` function; if `None` (the default), the models will be named according to their filenames in the results folder.

`consistent_colors` is a dictionary that attaches an index to each model name in order to ensure the same model is depicted using the same color in any line graphs. `consistent_colors` is provided as an optional parameter to the `visualize_sample_error` function; if `None` (the default), models will be assigned arbitrary colors that may vary between graphs.

In [ ]:
display_names = {'Baseline':'Baseline', 'Dual Concat Mask Only':'Dual<sub>mask</sub>', 'Dual Concat Map Only':'Dual<sub>map</sub>',
               'Dual Concat Mask then Map':'Dual<sub>mask-map</sub>', 'Dual Concat Map then Mask':'Dual<sub>map-mask</sub>',
               'Dual Concat Input':'Dual<sub>input</sub>', 'UNet Baseline':'Skip', 'UNet Concat Input':'Skip<sub>input</sub>',
               'UNet Concat Map Only':'Skip<sub>map</sub>','UNet Concat Mask Only':'Skip<sub>mask</sub>',
               'UNet Concat Map then Mask':'Skip<sub>map-mask</sub>', 'UNet Concat Mask then Map':'Skip<sub>mask-map</sub>',
               'UNet Concat Input': 'Skip<sub>input</sub>', 'ResUNet':'Skip<sub>residual</sub>'}
consistent_colors = {v[1]:v[0] for v in enumerate(display_names.values())}

# Results Analysis

## Average Error for All Models (Table)

In [ ]:
avg_dfs = get_average_error(all_results)
avg_dfs = avg_dfs.set_index('Model')
avg_dfs = avg_dfs.sort_values(['Avg Error'])
avg_dfs.index.name = None
avg_dfs

## Model Size Comparison (Table)

In [ ]:
mdl_names = avg_dfs.index

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

models = dict()
for name in mdl_names:
  model = torch.load(os.path.join(model_folder, name + '.pth'), map_location=device)
  models[name] = model

params = {'Encoder':[], 'Decoder':[], 'Total':[]}
for model in models.values():
  params['Encoder'].append(count_parameters(model.encoder))
  params['Decoder'].append(count_parameters(model.decoder))
  params['Total'].append(count_parameters(model))
  assert(params['Encoder'][-1] + params['Decoder'][-1] == params['Total'][-1])

params_df = pd.DataFrame.from_dict(params, orient='columns')
params_df.index = models.keys()
params_df = params_df.sort_values(['Total'])
params_df

## Model Size vs Performance

In [ ]:
df_vis = params_df.join(avg_dfs)
df_vis['TotalVis'] = df_vis['Total'] - df_vis['Total'].min() + 2000
df_vis['Text'] = (df_vis['Total'] // 1000).astype(str) + 'K'
df_vis['Model'] = df_vis.index
df_vis['Neg Error'] = df_vis['Avg Error'] * -1
df_vis=df_vis.sort_values(['Total', 'Neg Error'])
df_vis.reindex([])
df_vis = df_vis.replace({'Model': display_names})
df_vis['Colors'] = df_vis['Model'].copy()
df_vis.replace({'Colors': consistent_colors})

fig = px.scatter(df_vis, x="Model", y="Avg Error", size="TotalVis", color="Model", size_max=90, text='Text', labels=display_names)
fig.update_traces(textposition='top center')
fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(range=[1.64, 2.25])
fig.update_layout(showlegend=False, yaxis_title='RMSE(dB)')
fig.update_layout(width=1500, height=800)
fig.update_layout(font=dict(size=34))

fig.update_layout(shapes=[go.layout.Shape(type='rect', xref='paper', yref='paper', x0=0, y0=0, x1=1, y1=1, line={'width': 1, 'color': 'black', 'dash':'solid'})])
fig.update_xaxes(
  ticks="outside",
  tickson="labels",
  ticklen=15,
  title=None)
fig.update_yaxes(
    ticks="outside",
    tickson="labels",
    ticklen=15)

fig.show()
fig.write_image(os.path.join(viz_folder, 'All Models Size.pdf'))

## Dual Path Models Average Error

In [ ]:
dual_avg_df = get_average_error(dual_results)
dual_avg_df = dual_avg_df.sort_values('Avg Error', ascending=False)

# Full figure
fig = visualize_average_error(dual_avg_df, display_names=display_names, baseline_name='Baseline', 
                              width=700, height=450, text_size=24)
fig.write_image(os.path.join(viz_folder,'Dual Path Avg.pdf'))

# Zoomed in figure (used in paper)
fig = visualize_average_error(dual_avg_df, display_names=display_names, baseline_name='Baseline', 
                              width=700, height=450, text_size=24, y_range=[1.5, 2.4])
fig.write_image(os.path.join(viz_folder,'Dual Path Avg Zoom.pdf'))

## Dual Path Models per-Sampling Rate Error

In [ ]:
dashes = ['dash','solid']
markers = ['star', 'diamond', 'square']
line_styles = [(d, m) for m in markers for d in dashes]

# Full figure (used in paper)
fig = visualize_sample_error(dual_results, display_names=display_names, consistent_colors=consistent_colors, line_styles=line_styles, width=700, height=450, text_size=23, marker_size=10)
fig.write_image('Dual Path All.pdf')

# Zoomed in figure
fig = visualize_sample_error(dual_results, display_names=display_names, consistent_colors=consistent_colors, line_styles=line_styles, width=700, height=450, text_size=23, marker_size=10, y_range=[0.9, 3.3], x_range=[0,0.4])
fig.write_image(os.path.join(viz_folder, 'Dual Path All Zoom.pdf'))

## Dual Path Models Average Split by Sampling Rate

In [ ]:
# Figure not used in paper
fig = visualize_hist(dual_results, display_names=display_names, baseline_name='Baseline', 
                     text_size=23, width=700, height=450)
fig.write_image("Dual Path Bins.pdf")

# Figure not used in paper
fig = visualize_hist(dual_results, display_names=display_names, baseline_name='Baseline', 
                     text_size=23, width=700, height=450, y_range=[0.5, 3.2])
fig.write_image("Dual Path Bins Zoom.pdf")

## UNet Models Average Error

In [ ]:
unet_avg_df = get_average_error(unet_results)
unet_avg_df = unet_avg_df.sort_values('Avg Error', ascending=False)

# Figure not used in paper
fig = visualize_average_error(unet_avg_df, display_names=display_names, baseline_name='Baseline', 
                              width=700, height=450, text_size=24)
fig.write_image(os.path.join(viz_folder, 'UNet Avg.pdf'))

# Figure not used in paper
fig = visualize_average_error(unet_avg_df, display_names=display_names, baseline_name='Baseline', 
                              width=700, height=450, text_size=24, y_range=[1.5, 2.4])
fig.write_image(os.path.join(viz_folder,'UNet Avg Zoom.pdf'))

## UNet Models per-Sampling Rate Error

In [ ]:
dashes = ['solid', 'dash', 'dot']
markers = ['circle', 'square', 'diamond']
line_styles = [(d, m) for m in markers for d in dashes]

# Full figure (used in paper)
fig = visualize_sample_error(unet_results, display_names=display_names, consistent_colors=consistent_colors, line_styles=line_styles, text_size=24, width=700, height=450, marker_size=10)
fig.write_image(os.path.join(viz_folder,"Unet All.pdf"))

# Zoomed in figure
fig = visualize_sample_error(unet_results, display_names=display_names, consistent_colors=consistent_colors, line_styles=line_styles, text_size=24, width=700, height=450, marker_size=10, y_range=[0.9, 3.3], x_range=[0,0.4])
fig.write_image(os.path.join(viz_folder,"Unet All Zoom.pdf"))

## UNet Models Average Error Split by Sampling Rate

In [ ]:
# Full figure
fig = visualize_hist(unet_results, display_names=display_names, baseline_name='Baseline',
                     text_size=23, width=700, height=450)
fig.write_image("UNet Bins.pdf")

# Zoomed in figure (used in paper)
fig = visualize_hist(unet_results, display_names=display_names, baseline_name='Baseline',
                     text_size=23, width=700, height=450, y_range=[0.5, 3])
fig.write_image("UNet Bins Zoom.pdf")

## All Models Average Error

In [ ]:
vis_avg_df = avg_dfs.copy()
vis_avg_df['Model'] = vis_avg_df.index
vis_avg_df = vis_avg_df.sort_values('Avg Error', ascending=False)

# Figure not used in paper
fig = visualize_average_error(vis_avg_df, display_names=display_names, baseline_name='Baseline', 
                              width=1200, height=450, text_size=24)
fig.write_image(os.path.join(viz_folder, "All Models Average.pdf"))

## Dual Path vs UNet per Sampling Rate Error

To compare Dual Path and UNet models without overly cluttering the graph, we visualize just the median performing models of each group and the Baseline. In fact, we include two medians for the Dual Path group, those that pass the sampled map to the Decoder (Top) and those that pass the environment mask (Bottom), since there is significant difference between these groups.

To do this, we copy the median model performances to a new `group_folder` then pass this folder to the `visualize_sample_error` function along with a new `model_group_names` dictionary to rename each according to its group.

In [ ]:
group_folder = '/Path/to/group/folder'
model_group_names = {'Baseline':'Baseline', 'Dual Concat Mask Only':'Dual Path Models (Bottom)', 
                     'Dual Concat Mask then Map':'Dual Path Models (Top)', 'UNet Concat Mask Only':'Skip Connection Models'}

dashes = ['solid', 'dash', 'dot']
markers = ['circle', 'square', 'diamond', 'star']
line_styles = [(d, m) for d in dashes for m in markers]

# Full figure (used in paper)
fig = visualize_sample_error(group_folder, display_names=model_group_names, consistent_colors=consistent_colors, 
                             width=700, height=450, text_size=23, line_styles=line_styles, marker_size=10)
fig.write_image(os.path.join(viz_folder, "Model Groups All.pdf"))

# Zoomed in figure
fig = visualize_sample_error(group_folder, display_names=model_group_names, consistent_colors=consistent_colors, 
                             width=700, height=450, text_size=23, line_styles=line_styles, marker_size=10, y_range=[0.9, 3.3], x_range=[0, 0.4])
fig.write_image(os.path.join(viz_folder, "Model Groups All Zoom.pdf"))